In [2]:
import pandas as pd
import requests
import csv


def conver(i):
    if int(i) < 10: return "0" + str(i)
    else: return str(i)


# searDate = input("請輸入查詢年月日(格式:1999 01 01):").split(" ")
# date = searDate[0]+conver(searDate[1])+conver(searDate[2])

# stockY = input("請輸入查詢年(格式:1999):")
# stockM = input("請輸入查詢月(格式:01):")
# stockD = input("請輸入查詢日(格式:01):")
# date = stockY+conver(stockM)+conver(stockD)

# date = input("請輸入查詢年月日(格式:19990101):")
searMonth = input("請輸入查詢年月(格式:199901):")
for i in range(1, 32):

    html = requests.get(
        "http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=" +
        searMonth + conver(str(i)) + "&type=ALLBUT0999&_=1544760086323")

    with open(
            searMonth + conver(str(i)) + ".csv", "w",
            encoding="utf-8-sig") as fp:
        fp.write(html.text)
    with open(
            searMonth + conver(str(i)) + ".csv", "r",
            encoding="utf-8-sig") as fp:
        content = list(csv.reader(fp))

    # 找出content裡有17個欄位的元素，加到stockLi裡
    stockLi = []
    for j in content:
        if len(j) == 17:
            del j[-1]
            stockLi.append(j)

    if stockLi == []: continue

    # 刪除第 0 列
    try:
        if stockLi[0][0] != "證券代號":
            del stockLi[0]

        # 整理資料裡無用的符號，字串的replace()會成為新的元素，所以要重新寫回
        for k in range(len(stockLi)):
            stockLi[k][0] = stockLi[k][0].replace('"', '')
            stockLi[k][0] = stockLi[k][0].replace('=', '')
            for l in range(2, 16):
                stockLi[k][l] = stockLi[k][l].replace(',', '')

        # 將有多個 List的 List直接轉成 DataFrame
        from pandas.core.frame import DataFrame

        stockDf = pd.DataFrame(stockLi[1:], columns=stockLi[0])
        stockDf = stockDf.set_index("證券代號")
        stockDf
        # 找出有開盤且收益大於 1.05 的股票
        # for i in range(1,len(stockLi)):
        #     if stockLi[i][5] != '--' and (float(stockLi[i][8])/float(stockLi[i][5]))>1.05:
        #         print(stockLi[i])
        import sqlite3

        conn = sqlite3.connect("stock.db")

        stockDf.to_sql("data" + searMonth + conver(str(i)), conn)
        stockSql = pd.read_sql(
            "select * from data" + searMonth + conver(str(i)),
            conn,
            index_col=["證券代號"])
        print(stockSql.head())
        conn.close();
    except IndexError:
        print("非營業日")

請輸入查詢年月(格式:199901):201811


ValueError: Table 'data20181101' already exists.

In [3]:
conn.close();